In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

In [2]:
xgbo_path = '../results/pincher_xgb_baysian_results_1h.csv'
svr_path = '../results/pincher_svr_baysian_results_1h.csv'


In [3]:
xgb_df = pd.read_csv(xgbo_path)
svr_df = pd.read_csv(svr_path)

In [4]:
xgb_df.head()

,Unnamed: 0,target_wind_speed,pred_xgb
0,117753,5.0,5.509674
1,117754,0.0,6.559263
2,117755,9.0,6.045299
3,117756,11.0,10.005438
4,117757,11.0,9.638950


In [5]:
xgb_df.shape

(720, 3)

In [6]:
svr_df.head()

,Unnamed: 0,target_wind_speed,pred_svr
0,117753,5.0,4.322620
1,117754,0.0,5.345310
2,117755,9.0,4.232998
3,117756,11.0,7.911787
4,117757,11.0,10.167980


In [7]:
svr_df.shape

(720, 3)

In [8]:
# 1. Set 'Unnamed: 0' as the index in both DataFrames
xgb_df = xgb_df.set_index('Unnamed: 0')
svr_df = svr_df.set_index('Unnamed: 0')

# 2. Align both DataFrames to the intersection of their indices
#    (i.e., only keep rows that exist in BOTH)
common_index = xgb_df.index.intersection(svr_df.index)
xgb_aligned = xgb_df.loc[common_index]
svr_aligned = svr_df.loc[common_index]

# 3. Optional: Verify alignment
assert xgb_aligned.index.equals(svr_aligned.index), "Indices do not match!"

In [10]:
y_true = xgb_aligned.loc[common_index, 'target_wind_speed'].values
p_xgb  = xgb_aligned.loc[common_index, 'pred_xgb'].values
p_svr  = svr_aligned.loc[common_index, 'pred_svr'].values

In [11]:
print(f"Aligning Models... Found {len(y_true)} common hours for evaluation.")

Aligning Models... Found 720 common hours for evaluation.


In [12]:
# 2. FIND OPTIMAL BLEND WEIGHT
# We test weights from 0.00 to 1.00
best_mae = 9999
best_rmse = 9999
best_weight = 0

print("\nSearching for the perfect blend...")
results_list = []

for w in np.arange(0, 1.01, 0.01):
    # The Formula: (Weight * XGB) + ((1-Weight) * SVR)
    p_blend = (w * p_xgb) + ((1 - w) * p_svr)
    
    mae = mean_absolute_error(y_true, p_blend)
    rmse = np.sqrt(mean_squared_error(y_true, p_blend))
    
    results_list.append({'weight_xgb': w, 'mae': mae, 'rmse': rmse})
    
    if mae < best_mae:
        best_mae = mae
        best_rmse = rmse
        best_weight = w



Searching for the perfect blend...


In [13]:

# 3. PRINT FINAL RESULTS FOR YOUR PAPER
print("\n" + "="*40)
print(f"🏆 FINAL HYBRID MODEL RESULTS 🏆")
print("="*40)
print(f"Optimal Configuration:")
print(f"   XGBoost Weight: {best_weight:.2f} ({best_weight*100:.0f}%)")
print(f"   SVR Weight:     {1-best_weight:.2f} ({(1-best_weight)*100:.0f}%)")
print("-" * 40)
print(f"Hybrid MAE:  {best_mae:.5f}")
print(f"Hybrid RMSE: {best_rmse:.5f}")
print("="*40)



🏆 FINAL HYBRID MODEL RESULTS 🏆
Optimal Configuration:
   XGBoost Weight: 0.69 (69%)
   SVR Weight:     0.31 (31%)
----------------------------------------
Hybrid MAE:  6.30497
Hybrid RMSE: 8.85036


In [ ]:
import pandas as pd

# Convert results to DataFrame
results_df = pd.DataFrame(results_list)

# Save to CSV
results_df.to_csv('blend_weight_search_results.csv', index=False)

print(f"\nBest weight for XGBoost: {best_weight:.2f}")
print(f"Best MAE: {best_mae:.5f}, Best RMSE: {best_rmse:.5f}")
print("Results saved to 'blend_weight_search_results.csv'")